In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#Lets provide some insight into PO.DAAC impact metrics 
#in Elseviers SCOPUS service

In [3]:
#First lets import the libraries we require
from pprint import pprint
import podaac.podaac as podaac
import podaac.podaac_utils as utils

In [4]:
#Then we can create instances of the classes we will use
p = podaac.Podaac()
u = utils.PodaacUtils()

In [5]:
#Print a list of dataset short names
print('\nHeres list_all_available_granule_search_dataset_short_names()')
result = u.list_all_available_granule_search_dataset_short_names()
dsetShortName = [i for i in result]
print(len(result))


Heres list_all_available_granule_search_dataset_short_names()
607


In [6]:
#dataset_landing_pages = []
#Perform a landing page search on each dataset, note this can take some time as we are querying >600 times
#for ds in dsetShortName:
#    result = p.dataset_search(short_name=ds)
#    #Cache the dataset landing page URL
#    searchStr = 'http://podaac.jpl.nasa.gov/dataset/'
#    for i in result.strip().split():
#        if searchStr in i:
#            dataset_landing_pages.append(str(i))
#print(dataset_landing_pages)

In [7]:
from bs4 import BeautifulSoup
poddac_dois = []
#Perform a doi search on each dataset, note this can take some time as we are querying >600 times
for ds in dsetShortName:
    try:
        result = p.dataset_search(short_name=ds, full='True')
    except Exception:
        print('Error retrieving record for ' + ds)
    soup = BeautifulSoup(result, 'html.parser')
    doi_element = soup.find('podaac:datasetdoi', text=True)
    if doi_element is not None:
        doi = doi_element.text #do not include 'https://doi.org/' prefix as it eventually makes search less accurate and increases runtime
        poddac_dois.append(doi)

404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=atom&full=True&shortName=WAF_DEALIASED_SASS_L2
Error retrieving record for WAF_DEALIASED_SASS_L2
404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=atom&full=True&shortName=WENTZ_SASS_SIGMA0_L2
Error retrieving record for WENTZ_SASS_SIGMA0_L2
404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=atom&full=True&shortName=UCLA_DEALIASED_SASS_L3
Error retrieving record for UCLA_DEALIASED_SASS_L3
404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=atom&full=True&shortName=SEASAT_BYU_L3_OW_SIGMA0_ENHANCED
Error retrieving record for SEASAT_BYU_L3_OW_SIGMA0_ENHANCED
404 Client Error: Not Found for url: https://podaac.jpl.nasa.gov/ws/search/dataset/?itemsPerPage=7&pretty=True&format=

In [8]:
print(len(poddac_dois))

570


In [9]:
import json
import requests
data = []
full_dois = []
full_titles = []
doi_count = 0
#Using Elsevier's Scopus Search, lets see if we can 
#retreieve any information from the above dataset dois
for podaac_doi in poddac_dois:
    url = 'https://api.elsevier.com/content/search/scopus?query=ALL:' + podaac_doi + '&date=2017-2019&APIKey=715b412c00f0b95e918a3e7abe6e6ee4'
    try:
        metadata = requests.get(url)
        status_codes = [404, 400, 503, 408]
        if metadata.status_code in status_codes:
            metadata.raise_for_status()
    except requests.exceptions.HTTPError as error:
        print(error)
        raise
    json_data = json.loads(metadata.text)
    doi_count = doi_count + int(json_data['search-results']['opensearch:totalResults'])
    entries = json_data['search-results']['entry']
    for entry in entries:
        if 'error' not in entry:
            try:
                prism_doi = entry['prism:doi']
            except KeyError:
                prism_doi = '.'
            full_dois.append(prism_doi)
            try:
                prism_publication_name = entry['prism:publicationName']
            except KeyError:
                prism_publication_name = '.'
            try:
                dc_title = entry['dc:title']
            except KeyError:
                dc_title = '.'
            full_titles.append(dc_title.lower())
            try:
                citedby_count = entry['citedby-count']
            except KeyError:
                citedby_count = '.'
            data.append(['.', '.', dc_title, prism_publication_name, citedby_count, entry['prism:url'], prism_doi, '.', '.', '.', '.', '.', '.', '.', '.'])
print('Located {} PO.DAAC DOI hits from Scopus database.'.format(doi_count))

Located 76 PO.DAAC DOI hits from Scopus database.


In [10]:
#Again, using Elsevier's Scopus Search, lets see other 'podaac' resources we can retreive.
orig_url = 'https://api.elsevier.com/content/search/scopus?query=ALL:podaac&date=2017-2019&APIKey=715b412c00f0b95e918a3e7abe6e6ee4'
def process_url(url):
    try:
        metadata = requests.get(url)
        status_codes = [404, 400, 503, 408]
        if metadata.status_code in status_codes:
            metadata.raise_for_status()
    except requests.exceptions.HTTPError as error:
        print(error)
        raise
    json_data = json.loads(metadata.text)
    if (json_data['search-results']['link'][2]['@ref'] or json_data['search-results']['link'][3]['@ref'] == 'next') or (json_data['search-results']['link'][0]['@hef'] == json_data['search-results']['link'][3]['@hef']):
        switch = ''
        if json_data['search-results']['link'][2]['@ref'] == 'next':
            switch = 'first'
        elif json_data['search-results']['link'][3]['@ref'] == 'next':
            switch = 'pagination'
        else:
            switch = 'last'
        entries = json_data['search-results']['entry']
        for entry in entries:
            try:
                prism_doi = entry['prism:doi']
                full_dois.append(prism_doi)
            except KeyError:
                prism_doi = '.'
            try:
                prism_publication_name = entry['prism:publicationName']
            except KeyError:
                prism_publication_name = '.'
            try:
                dc_title = entry['dc:title']
                full_titles.append(dc_title.lower())
            except KeyError:
                dc_title = '.'
            try:
                citedby_count = entry['citedby-count']
            except KeyError:
                citedby_count = '.'
            data.append(['.', '.', dc_title, prism_publication_name, citedby_count, entry['prism:url'], prism_doi, '.', '.', '.', '.', '.', '.', '.', '.'])
        if switch == 'first':
            print('Processing: ' + json_data['search-results']['link'][2]['@href'])
            process_url(json_data['search-results']['link'][2]['@href'])
        elif switch == 'pagination':
            print('Processing: ' + json_data['search-results']['link'][3]['@href'])
            process_url(json_data['search-results']['link'][3]['@href'])
        elif switch == 'last':
            return
process_url(orig_url)
#doi_count = doi_count + process_url(url, doi_count)
#print('Revised DOI hits to {} after "podaac" keyword search in Scopus database.'.format(doi_count))
#print(full_dois)

Processing: https://api.elsevier.com/content/search/scopus?start=25&count=25&query=ALL%3Apodaac&date=2017-2019&apiKey=715b412c00f0b95e918a3e7abe6e6ee4
Processing: https://api.elsevier.com/content/search/scopus?start=50&count=25&query=ALL%3Apodaac&date=2017-2019&apiKey=715b412c00f0b95e918a3e7abe6e6ee4
Processing: https://api.elsevier.com/content/search/scopus?start=75&count=25&query=ALL%3Apodaac&date=2017-2019&apiKey=715b412c00f0b95e918a3e7abe6e6ee4
Processing: https://api.elsevier.com/content/search/scopus?start=100&count=25&query=ALL%3Apodaac&date=2017-2019&apiKey=715b412c00f0b95e918a3e7abe6e6ee4


In [11]:
print(len(full_dois))

177


In [12]:
import pandas as pd
#Simply print the data as a pandas data frame
pd.DataFrame(data, columns=["Dataset Name as Cited in Article", "PERSISTENT_ID", "Manuscript Title", "Journal Source", "Impact factor of the Manuscript", "Article URL", "DOI", "PDF Name", "In-Text", "In-Text 2", "Referenced", "Referenced 2", "In-Text", "Referenced", "Confirmed"])

,Dataset Name as Cited in Article,PERSISTENT_ID,Manuscript Title,Journal Source,Impact factor of the Manuscript,Article URL,DOI,PDF Name,In-Text,In-Text 2,Referenced,Referenced 2,In-Text,Referenced,Confirmed
0,.,.,Covariability of near-surface wind speed stati...,Journal of Physical Oceanography,0,https://api.elsevier.com/content/abstract/scop...,10.1175/JPO-D-17-0177.1,.,.,.,.,.,.,.,.
1,.,.,"Intercomparison of Antarctic ice-shelf, ocean,...",Geoscientific Model Development,1,https://api.elsevier.com/content/abstract/scop...,10.5194/gmd-11-1257-2018,.,.,.,.,.,.,.,.
2,.,.,Evaluation of satellite-derived SST products i...,Marine Technology Society Journal,2,https://api.elsevier.com/content/abstract/scop...,10.4031/MTSJ.52.3.7,.,.,.,.,.,.,.,.
3,.,.,Revealing the full ocean migration of individu...,Animal Biotelemetry,2,https://api.elsevier.com/content/abstract/scop...,10.1186/s40317-018-0146-2,.,.,.,.,.,.,.,.
4,.,.,Impact of SST on heavy rainfall events on east...,Atmospheric Research,3,https://api.elsevier.com/content/abstract/scop...,10.1016/j.atmosres.2017.09.019,.,.,.,.,.,.,.,.
5,.,.,Impact of land-cover change between 1990 and 2...,Hydrological Research Letters,0,https://api.elsevier.com/content/abstract/scop...,10.3178/hrl.11.187,.,.,.,.,.,.,.,.
6,.,.,High precision dynamic orbit integration for s...,Advances in Space Research,1,https://api.elsevier.com/content/abstract/scop...,10.1016/j.asr.2017.04.015,.,.,.,.,.,.,.,.
7,.,.,Representing species distributions in spatiall...,Fisheries Research,1,https://api.elsevier.com/content/abstract/scop...,10.1016/j.fishres.2018.10.011,.,.,.,.,.,.,.,.
8,.,.,Numerical low-rank approximation of matrix dif...,Journal of Computational and Applied Mathematics,2,https://api.elsevier.com/content/abstract/scop...,10.1016/j.cam.2018.01.035,.,.,.,.,.,.,.,.
9,.,.,Argo Observations of the Deep Mixing Band in t...,Journal of Geophysical Research: Oceans,1,https://api.elsevier.com/content/abstract/scop...,10.1029/2018JC014275,.,.,.,.,.,.,.,.


In [13]:
#Lets find the overlap/match between what's been retreived from SCOPUS and what the JPL Librarians discovered
#We can do this by attemtping matches on DOI
from openpyxl import load_workbook
jpl_doi_list = []
wb = load_workbook(filename = '2017_PO.DAAC_Citation_cleaned_and_verified.xlsx', read_only=True)
ws = wb['Cleaned']
for doi_cell in ws['G2':'G169']:
    for doi in doi_cell:
        doi_str = str(doi.value)
        jpl_doi_list.append(doi_str.replace('https://doi.org/',''))
print('Matched DOI Hits: ' + str(len([i for i in jpl_doi_list if i in full_dois])))

Matched DOI Hits: 46


In [14]:
#Trying the same with Manuscript titles is not reliable... the JPL versions are not 'clean' enough.
jpl_title_list = []
wb = load_workbook(filename = '2017_PO.DAAC_Citation_cleaned_and_verified.xlsx', read_only=True)
ws = wb['Cleaned']
for title_cell in ws['C2':'C169']:
    for title in title_cell:
        title_str = str(title.value)
        jpl_doi_list.append(doi_str.lower())
title_match_count = 0
print('Matched DOI Hits: ' + str(len([i for i in jpl_title_list if i in full_titles])))

Matched DOI Hits: 0
